In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))

# Prepared biomarker data

In [ ]:
ready <- 
fread(paste0(SHARE_DIR, "biomarkers_base.csv")) %>% 
 fi(!is.na(purity), !is.na(durableClinicalBenefit)) %>% 
 mu(nrBor = abs(bestOverallResponse-1), nrDcb = abs(durableClinicalBenefit-1)) 

In [ ]:
K <- 70

# Updating features

- Select features by tool

In [ ]:
base_features <- 
names(ready %>% se( contains("cider_"), contains("clin_"), contains("cn_"), contains("driver"), 
                    contains("fusion"), contains("gie_"), contains("lilac_"), contains("neo_"), contains("chord"), 
                    contains("purity"), contains("rna_"), contains("sv_"), contains("teal_"), contains("viral_"), 
                    contains("hotspot"), contains("bacterial"), contains("teal"), contains("signature")))

- Feature filtering

In [ ]:
filter_ref <- 
data.frame("mn" = apply(ready %>% se(any_of(base_features)), 2, mean, na.rm = TRUE), 
   "sd" = apply(ready %>% se(any_of(base_features)), 2, sd, na.rm = TRUE), 
   "zeros" = apply(ready %>% se(any_of(base_features)) == 0, 2, sum, na.rm = TRUE),
   "non_zeros" = apply(ready %>% se(any_of(base_features)) != 0, 2, sum, na.rm = TRUE), 
   "nas" = apply(is.na(ready %>% se(any_of(base_features))), 2, sum, na.rm = TRUE), 
   "non_nas" = apply(!is.na(ready %>% se(any_of(base_features))), 2, sum, na.rm = TRUE)) %>% 
 mu(pct_zeros = zeros/(zeros+non_zeros), pct_nas = nas/(nas+non_nas))

- Rules

In [ ]:
base_features <- rownames(filter_ref %>% fi(pct_zeros < .99, non_nas > 100))
bin_features <- ready %>% se(all_of(base_features)) %>% select(where(~all(. %in% c(0, 1, NA))))
non_bin_features <- ready %>% se(all_of(base_features)) %>% select(!where(~all(. %in% c(0, 1, NA)))) 

- Functions to make features binary

In [ ]:
integerer <- function(df){
 df[] <- lapply(df, function(x) if(is.logical(x)) as.integer(x) else x); 
 df    
}

In [ ]:
binarify <- function(df, threshold = 50, direction = "gt" ){
 if(direction == "gt"){
   tmp <- df %>% 
    mu(across(everything(), ~ (. > quantile(., threshold/100, na.rm = TRUE)))) %>% 
    rename_with(~ paste0(.x, "_gt", as.character(threshold)))
 } else {
   tmp <- df %>% 
    mu(across(everything(), ~ (. < quantile(., threshold/100, na.rm = TRUE)))) %>% 
    rename_with(~ paste0(.x, "_lt", (as.character(threshold))))
 }
 integerer(tmp)
}

# Bin the continuous features into categorical

- Compute greater than 50 and extract `smooth` features that could be futher refined.

In [ ]:
gt50 <- binarify(non_bin_features, 50, "gt")
lt50 <- binarify(non_bin_features, 50, "lt")

In [ ]:
smooth_features <- 
unlist(lapply(names(Filter(function(x) .48 < x && x < .52, apply(gt50, 2, mean, na.rm = TRUE))), 
                           function(i) strsplit(i, "_gt50")[[1]][1]))

In [ ]:
gt25 <- binarify(non_bin_features %>% se(any_of(smooth_features)), 25, "gt")
gt75 <- binarify(non_bin_features %>% se(any_of(smooth_features)), 75, "gt")
lt25 <- binarify(non_bin_features %>% se(any_of(smooth_features)), 25, "lt")
lt75 <- binarify(non_bin_features %>% se(any_of(smooth_features)), 75, "lt")

# Combine features and save

In [ ]:
features_ready <- cbind(bin_features, gt50, lt50, gt25, gt75, lt25, lt75)

In [ ]:
ready <- cbind(ready %>% se(-all_of(base_features)), features_ready)

# Write Features

In [ ]:
saveRDS( list("ready" = ready, "features" = names(features_ready)), paste0(SHARE_DIR, "biomarkers_ready.Rds"))